<a href="https://colab.research.google.com/github/peppefdf/CSL_Gipuzkoa/blob/main/DragDrop_Markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
#%pip install osmnx
#%pip install dash
#%pip install dash_leaflet
#%pip install dash_bootstrap_components

import dash
from dash import Dash
import dash_bootstrap_components as dbc
#import dash_html_components as html
from dash import html, callback_context, ALL
from dash import dcc, Output, Input, State, callback
import dash_leaflet as dl
import re
import json
import pandas as pd


from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

im1 = '/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/CSL_logo.PNG'
im2 = '/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/DFG_logo.png'
from PIL import Image
image1 = Image.open(im1)
image2 = Image.open(im2)

center = (43.26852347667122, -1.9741372404905988)
#    iconUrl= 'https://uxwing.com/wp-content/themes/uxwing/download/location-travel-map/bus-stop-icon.png',
#    iconUrl= "https://i.ibb.co/6n1tzcQ/bus-stop.png",
custom_icon = dict(
    iconUrl= "https://i.ibb.co/HV0K5Fp/bus-stop.png",
    iconSize=[40,40],
    iconAnchor=[22, 40]
)


app = Dash(prevent_initial_callbacks=True)
app.layout = html.Div([
    html.Img(src=image1,style={'width':'40%', "display": "inlineBlock", "verticalAlign": "top"}),
    html.Img(src=image2,style={'width':'25%',"display": "inlineBlock", "verticalAlign": "top"}),
    html.Div(dl.Map([dl.TileLayer()],
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})),
    dbc.Button("Load stops", id="load_stops", n_clicks=0),
    html.Div(id='clickdata'),
    dcc.Store(id='internal-value', data=[])
])

#@app.callback([Output("clickdata", "children")],
@app.callback([Output("clickdata", "children"), Output('internal-value','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [Input("load_stops", "n_clicks")]
              )
def load_stops(N):
    filename = '/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/Accessibility_Map/INPUT_stops.csv'
    df = pd.read_csv(filename)
    #out=St.loc[:'Lat']
    #for i in range(len(St)):
    #    out = out + str(St.loc[i,['Lat']]) + ', ' + str(St.loc[i,['Lon']]) + '; '
    out = ''
    St = []
    for ind in df.index:
         out = out + str(df['Lat'][ind]) + ',' + str(df['Lon'][ind]) +';'
         St.append((df['Lat'][ind],df['Lon'][ind]))
    markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    #return [out,St,newMap]
    return [out,St,newMap]


@app.callback([Output("clickdata", "children",allow_duplicate=True), Output('internal-value','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [State('internal-value','data')],
              [Input('map','clickData')]
              )
def add_marker(St,clickd):
    marker_lat = clickd['latlng']['lat']
    marker_lon = clickd['latlng']['lng']
    St.append((marker_lat,marker_lon))
    out=''
    for i in range(len(St)):
        out = out + str(St[i][0]) + ', ' + str(St[i][1]) + '; '
    #markers = [dl.Marker(dl.Tooltip("double click on Marker after drag to update its position"), position=pos, id="marker{}".format(i), draggable=True) for i, pos in enumerate(St)]
    #Layer_group = dl.LayerGroup(markers, id="markers_group")
    #Layer_group = dl.LayerGroup(markers, id={"type": "markers_group", "index": 0})
    #markers = [dl.Marker(id={'type': 'marker', 'index': key}, position=data[key]) for key in data]
    markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [out,St,newMap]

@app.callback([Output("clickdata", "children",allow_duplicate=True),Output('internal-value','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [State('internal-value','data')],
              [Input({"type": "marker", "index": ALL},"n_clicks")],
              prevent_initial_callbacks=True)
def remove_marker(St,*args):
    marker_id = callback_context.triggered[0]["prop_id"].split(".")[0].split(":")[1].split(",")[0]
    n_clicks = callback_context.triggered[0]["value"]
    if n_clicks ==2:
       del St[int(marker_id)]
    out=''
    for i in range(len(St)):
        out = out + str(St[i][0]) + ', ' + str(St[i][1]) + '; '
    markers = [dl.Marker(dl.Tooltip("Double click on Marker to remove it"), position=pos, icon=custom_icon, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=center, zoom=12, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [out,St,newMap]


if __name__ == '__main__':
    app.run_server(Debug=True)

Mounted at /content/drive


<IPython.core.display.Javascript object>